# Test the endpoint

In [1]:
import boto3

client = boto3.client('sagemaker-runtime')

In [ ]:

# An example of a trace ID.
custom_attributes = "c000b4f9-df62-4c85-a0bf-7c525f9104a4"

endpoint_name = "10-2023-03-27-11-50-07-087"
content_type = "application/json"
accept = "application/json"
payload = "{'inference':['this is a test]}"
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    # CustomAttributes=custom_attributes,
    ContentType=content_type,
    Accept=accept,
    Body=payload,
)


In [7]:
import os
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [8]:
class MyTokenizer:
    def __init__(self, model_name="distilbert-base-uncased") -> None:
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

    def tokenize(self, txt_input):
        return self.tokenizer.encode(txt_input, padding="max_length", truncation=True)

In [9]:
tok = MyTokenizer()

In [11]:
tok.tokenizer(["htis is a test"])

{'input_ids': [[101, 1044, 7315, 2003, 1037, 3231, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}